In [4]:
import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose

from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet

from tqdm.notebook import tqdm


def process_video_with_depth_estimation(video_path, outdir, device, margin_width=50, duration_seconds=30):

    depth_anything = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format('vitl')).to(DEVICE).eval()

    transform = Compose([
        Resize(
            width=518,
            height=518,
            resize_target=False,
            keep_aspect_ratio=True,
            ensure_multiple_of=14,
            resize_method='lower_bound',
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        PrepareForNet(),
    ])

    try: 
        os.makedirs(outdir, exist_ok=True)

        video = cv2.VideoCapture(video_path)
        frame_width, frame_height = int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(video.get(cv2.CAP_PROP_FPS))
        total_frames_to_process = int(fps * duration_seconds)

        output_path = os.path.join(outdir, os.path.basename(video_path).replace(".m4v", "_video_depth.mp4"))
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width + margin_width, frame_height))


        for _ in tqdm(range(total_frames_to_process), desc="Processing video"):
            ret, frame = video.read()
            if not ret:
                break

            raw_l_img = frame[:, :int(frame_width / 2)]

            l_img = cv2.cvtColor(raw_l_img, cv2.COLOR_BGR2RGB) / 255.0
            l_img = transform({'image': l_img})['image']
            l_img = torch.from_numpy(l_img).unsqueeze(0).to(device)

            with torch.no_grad():
                depth = depth_anything(l_img)

            depth = F.interpolate(depth.unsqueeze(0), (raw_l_img.shape[0], raw_l_img.shape[1]), mode='bilinear', align_corners=False).squeeze()
            depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
            depth = depth.cpu().numpy().astype(np.uint8)
            depth_color = cv2.applyColorMap(depth, cv2.COLORMAP_INFERNO)

            split_region = np.ones((frame_height, margin_width, 3), dtype=np.uint8) * 255
            combined_frame = cv2.hconcat([raw_l_img, split_region, depth_color])

            out.write(combined_frame)


    except KeyboardInterrupt:
        print("Interrupted by user, cleaning up")
    finally: 
        video.release()
        out.release()
        print("Resources have been released successfully.")

# Usage
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
VIDEO_PATH = "/home/emilia/MasterOfFish/sintef_data/YC1/1.m4v"
OUTDIR = "/home/emilia/MasterOfFish/sintef_data/YC1/depthAny"
process_video_with_depth_estimation(VIDEO_PATH, OUTDIR, DEVICE, duration_seconds=30)


config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

xFormers not available
xFormers not available


Processing video:   0%|          | 0/750 [00:00<?, ?it/s]

Resources have been released successfully.
